In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import os
import re
import cv2
import numpy as np
from torchvision import transforms
import random

# ensure determinism
seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * 147 * 147, 256)  # Adjusted input size
        self.fc2 = nn.Linear(256, 2) # predicting two variables (alfa x and betax)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class CustomDataset(Dataset):
    def __init__(self, image_dir):
        self.image_dir = image_dir
        self.image_files = sorted(os.listdir(image_dir))
        self.pattern = r"epsnx([\d.-]+)_alfax([\d.-]+)_betax([\d.-]+)_epsny([\d.-]+)_alfay([\d.-]+)_betay([\d.-]+)_epsnz([\d.-]+)_alfaz([\d.-]+)_betaz([\d.-]+)\.png"
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, index):
        image_name = self.image_files[index]
        image_path = os.path.join(self.image_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.transform(image)
        matches = re.search(self.pattern, image_name)
        variables = [float(matches.group(i)) for i in range(1, 10) if matches.group(i)]
        alfa_x = variables[1] # alfax value
        beta_x = variables[2] # betax value
        return image, torch.tensor([alfa_x, beta_x]) # returns alfa x and betax as the labels




In [7]:
import torch
import os
from torch.utils.data import DataLoader
import math
import numpy as np

def evaluate_model(model, test_dir):
    model.eval()
    test_dataset = CustomDataset(test_dir)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    total_mae = torch.zeros(2, device=device)   # Added device argument
    total_mape = torch.zeros(2, device=device)  # Added device argument
    total_smape = torch.zeros(2, device=device) # Added device argument
    total_mse = torch.zeros(2, device=device)   # Added device argument
    total_count = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)

            absolute_error = torch.abs(outputs - labels)  # Absolute error
            total_mae += absolute_error.sum(dim=0)

            non_zero_mask = torch.abs(labels) > 1e-8  # Only calculate percentage for non-zero labels
            percentage_error = (absolute_error / torch.abs(labels)) * 100  # Percentage error
            total_mape += (percentage_error * non_zero_mask).sum(dim=0)

            smape = 200.0 * torch.abs(outputs - labels) / (torch.abs(outputs) + torch.abs(labels) + torch.finfo(torch.float32).eps)
            total_smape += smape.sum(dim=0)
            
            mse = (outputs - labels) ** 2  # Mean Square Error
            total_mse += mse.sum(dim=0)

            total_count += labels.size(0)  # number of samples

    mae = total_mae / total_count
    mape = total_mape / total_count
    smape = total_smape / total_count
    rmse = torch.sqrt(total_mse / total_count)

    return mae.cpu().numpy(), mape.cpu().numpy(), smape.cpu().numpy(), rmse.cpu().numpy()  # Moved tensors to CPU and converted to numpy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN()
model.load_state_dict(torch.load("models/cnn_oldmodel_500images_b4_e500.pth"))
model.to(device)

mae, mape, smape, rmse = evaluate_model(model, "test")
for i in range(2):
    print(f"Metrics for prediction {i+1}:")
    print(f"Mean Absolute Error: {mae[i]:.4f}")
    print(f"Mean Absolute Percentage Error: {mape[i]:.2f}%")
    print(f"Symmetric Mean Absolute Percentage Error: {smape[i]:.2f}%")
    print(f"Root Mean Square Error: {rmse[i]:.4f}")
    print()


Metrics for prediction 1:
Mean Absolute Error: 1.1893
Mean Absolute Percentage Error: nan%
Symmetric Mean Absolute Percentage Error: 81.21%
Root Mean Square Error: 1.7129

Metrics for prediction 2:
Mean Absolute Error: 14.0494
Mean Absolute Percentage Error: 6.39%
Symmetric Mean Absolute Percentage Error: 6.26%
Root Mean Square Error: 28.5530

